# Core Imports

In [1]:
# Custom Imports
from polysaccharide import general
from polysaccharide.molutils.rdmol import rdconvert, rdkdraw, rdbond, rdlabels
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer import building

import mbuild as mb

# File I/O
from pathlib import Path

# Cheminformatics
from rdkit import Chem

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Wa

# File and chemistry type definitions

In [2]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

omm_path = Path('OpenMM')
omm_path.mkdir(exist_ok=True)

In [3]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Building PDB files from monomer fragments

In [ ]:
from polysaccharide.polymer.exceptions import SubstructMatchFailedError

DOP = 10

failed = general.RecursiveDict()
for chemistry in chemistries:
    mono_src_dir = MONO_INFO_DIR / chemistry
    pdb_out_dir = pdb_path / chemistry
    pdb_out_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        mono_info = MonomerInfo.from_file(mono_path)
        try:
            polymer = building.build_linear_polymer(mono_info.SMARTS, DOP=DOP, sequence='AB')  
            savepath = pdb_out_dir / f'{mono_path.stem}.pdb'
            polymer.save(general.asstrpath(savepath))
        except Exception as e:
            failed[chemistry][e.__class__.__name__][mono_path.stem] = mono_info

In [ ]:
failed

## Testing sanitization of monomers prior to mbuild conversion

In [ ]:
rdkdraw.set_rdkdraw_size(500, 3/2)
cvtr = rdconvert.SMILESConverter()

chemistry = 'urethane'
idx = 10

mono_info = MonomerInfo.from_file(MONO_INFO_DIR / chemistry / f'{chemistry}_{idx}.json')

mbmols = {}
def show(mono_info : MonomerInfo) -> None:
    for resname, smarts in mono_info.SMARTS.items():
        print(resname, smarts)
        mol = Chem.MolFromSmarts(smarts)
        mol = cvtr.convert(mol)
        Chem.SanitizeMol(mol)
        rdlabels.clear_atom_map_nums(mol, in_place=True)
        display(mol)

        # mbmol, ports = building.mbmol_from_mono_smarts(new_smarts)

        prot_mol = rdbond.hydrogenate_rdmol_ports(mol, in_place=False)
        mbmols[resname] = mb.conversion.from_rdkit(prot_mol)

show(mono_info)